# Assignment 1 - web server

In this assignment, you will develop a simple Web server in Python that is capable
of processing only one request. Specifically, your Web server will 

(i) create a connection socket when contacted by a client (browser); 

(ii) receive the HTTP request from this connection;

(iii) parse the request to determine the specific file being requested; 

(iv) get the requested file from the server’s file system; 

(v) create an HTTP response message consisting of the requested file preceded by header lines;
and 

(vi) send the response over the TCP connection to the requesting browser. If a
browser requests a file that is not present in your server, your server should return a
“404 Not Found” error message.
In the companion Web site, we provide the skeleton code for your server. Your
job is to complete the code, run your server, and then test your server by sending
requests from browsers running on different hosts. If you run your server on a host
that already has a Web server running on it, then you should use a different port than
port 80 for your Web server.

In [0]:
import io
import socket
import json
import mimetypes
hostname = socket.gethostname()
port = 8000

In [0]:
def resolve_request(request):
    request_line, sep, request = request.partition(b'\r\n')
    method, url, version = request_line.split(b' ')

    return (url, version)

  
def generate_response(version, status_code, phrase, content_type, file_contents):
    status_line = version + b' ' + ' '.join([status_code, phrase]).encode('ASCII')
    connection_line = b'Connection: close'
    content_length_line = ('Content-Length: ' + str(len(file_contents))).encode('ASCII')
    content_type_line = ('Content-Type: ' + content_type).encode('ASCII')
    response_body = file_contents
    
    return b'\r\n'.join([status_line, connection_line, content_length_line, content_type_line + b'\r\n', response_body])


In [70]:
server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM) # create a TCP socket

server_sock.bind((hostname,port)) 
server_sock.listen(2)

print(server_sock)
while 1:
  print('---Listening---\n')

  connect_sock, address = server_sock.accept()
  request = connect_sock.recv(4096)

  print('---Request comes from IP: ' + str(address[0]))

  # ignore empty request
  if len(request) == 0:
      print('---Empty request, ignore')
      print('')
      continue

  # resolve the request
  url, version = resolve_request(request)
  path = '/content/gdrive/My Drive/Colab Notebooks/socketProgramming/session1/' + url.decode('ASCII')

  print('---Requested file path: ' + path)

  # try to open the file in the filesystem
  try:
      file_contents = open(path, 'rb').read()
      status_code = '200'
      phrase = 'OK'
      content_type = mimetypes.guess_type(path)[0]
      if not content_type:
          content_type = 'text/plain'            
      print('---File ' + path + ' found, generate 200 OK response')
  except IOError:
      file_contents = b'404: Page Not Found - by Burgess Wong'
      status_code = '404'
      phrase = 'Not Found'
      content_type = 'text/html'
      print('---File ' + path + ' not found, generate 404 Not Found response')

  # make the response message
  response = generate_response(version, status_code, phrase, content_type, file_contents)

  # send the response and close the socket
  connect_sock.send(response)

  print('---Response sent')

  connect_sock.close()

  print('---Connection with IP ' + str(address[0]) + ' closed\n')

<socket.socket fd=80, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('172.28.0.2', 8000)>
---Listening---

---Request comes from IP: 172.28.0.2
---Requested file path: /content/gdrive/My Drive/Colab Notebooks/socketProgramming/session1//hello.html
---File /content/gdrive/My Drive/Colab Notebooks/socketProgramming/session1//hello.html found, generate 200 OK response
---Response sent
---Connection with IP 172.28.0.2 closed

---Listening---



KeyboardInterrupt: ignored